**Image to text conversion**

In [ ]:
import base64
# Read the image file
with open("/content/apple.jpeg", "rb") as image_file:
    # Encode the image in Base64 format
    base64_image = base64.b64encode(image_file.read())

# Save the Base64 encoded data to a text file
with open("image_base64.txt", "w") as text_file:
    text_file.write(base64_image.decode("utf-8"))

**Text compression using Huffman coding(Ignore this part for now)**

In [ ]:
import heapq
from collections import Counter

class HuffmanCoding:
    class Node:
        def __init__(self, char, freq):
            self.char = char
            self.freq = freq
            self.left = None
            self.right = None

        def __lt__(self, other):
            return self.freq < other.freq

    def __init__(self, text):
        self.text = text
        self.frequency = Counter(text)
        self.heap = []
        self.codes = {}
        self.reverse_codes = {}

    def build_heap(self):
        for char, freq in self.frequency.items():
            node = self.Node(char, freq)
            heapq.heappush(self.heap, node)

    def merge_nodes(self):
        while len(self.heap) > 1:
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)
            merged = self.Node(None, node1.freq + node2.freq)
            merged.left = node1
            merged.right = node2
            heapq.heappush(self.heap, merged)

    def build_codes_helper(self, node, current_code):
        if node is None:
            return
        if node.char is not None:
            self.codes[node.char] = current_code
            self.reverse_codes[current_code] = node.char
        self.build_codes_helper(node.left, current_code + "0")
        self.build_codes_helper(node.right, current_code + "1")

    def build_codes(self):
        root = heapq.heappop(self.heap)
        self.build_codes_helper(root, "")

    def get_encoded_text(self):
        encoded_text = ""
        for char in self.text:
            encoded_text += self.codes[char]
        return encoded_text

    def pad_encoded_text(self, encoded_text):
        extra_padding = 8 - len(encoded_text) % 8
        for i in range(extra_padding):
            encoded_text += "0"
        padded_info = "{0:08b}".format(extra_padding)
        encoded_text = padded_info + encoded_text
        return encoded_text

    def get_byte_array(self, padded_encoded_text):
        if len(padded_encoded_text) % 8 != 0:
            print("Encoded text not padded properly")
            exit(0)
        byte_array = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            byte_array.append(int(byte, 2))
        return byte_array

    def compress(self, output_path):
        self.build_heap()
        self.merge_nodes()
        self.build_codes()
        encoded_text = self.get_encoded_text()
        padded_encoded_text = self.pad_encoded_text(encoded_text)
        byte_array = self.get_byte_array(padded_encoded_text)

        with open(output_path, 'wb') as output:
            # Store the frequency dictionary length
            output.write(len(self.frequency).to_bytes(2, 'big'))
            # Store the frequency dictionary
            for char, freq in self.frequency.items():
                output.write(char.encode('utf-8'))
                output.write(freq.to_bytes(4, 'big'))
            # Store the encoded text
            output.write(byte_array)

        return output_path

    def remove_padding(self, padded_encoded_text):
        padded_info = padded_encoded_text[:8]
        extra_padding = int(padded_info, 2)
        padded_encoded_text = padded_encoded_text[8:]
        encoded_text = padded_encoded_text[:-1*extra_padding]
        return encoded_text

    def decode_text(self, encoded_text):
        current_code = ""
        decoded_text = ""

        for bit in encoded_text:
            current_code += bit
            if current_code in self.reverse_codes:
                character = self.reverse_codes[current_code]
                decoded_text += character
                current_code = ""
        return decoded_text

    def decompress(self, input_path, output_path):
        with open(input_path, 'rb') as file:
            # Read the frequency dictionary length
            freq_dict_length = int.from_bytes(file.read(2), 'big')
            # Read the frequency dictionary
            frequency = {}
            for _ in range(freq_dict_length):
                char = file.read(1).decode('utf-8')
                freq = int.from_bytes(file.read(4), 'big')
                frequency[char] = freq

            # Build the Huffman tree from the frequency dictionary
            self.frequency = frequency
            self.build_heap()
            self.merge_nodes()
            self.build_codes()

            # Read the encoded text
            bit_string = ""
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)

        encoded_text = self.remove_padding(bit_string)
        decompressed_text = self.decode_text(encoded_text)

        with open(output_path, 'w') as output:
            output.write(decompressed_text)

        return output_path

# Example usage:
input_file_path = 'image_base64.txt'
output_compressed_path = 'compressed.txt'
# output_decompressed_path = 'decompressed.txt'

# Read the content of the provided text file
with open(input_file_path, 'r') as file:
    text = file.read()

# Create a HuffmanCoding instance
huffman = HuffmanCoding(text)

# Compress the text file
huffman.compress(output_compressed_path)

# Decompress the text file
# huffman.decompress(output_compressed_path, output_decompressed_path)

'compressed.txt'

**Key generation using ECDH**

In [ ]:
# Communication using ECDH

import hashlib
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

class ECDH:
    def __init__(self):
        self.private_key = ec.generate_private_key(ec.SECP384R1(), default_backend())
        self.public_key = self.private_key.public_key()
        self.shared_key = None

    def get_public_key_bytes(self):
        return self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

    def load_peer_public_key(self, peer_public_key_bytes):
        self.peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())

    def derive_shared_secret(self):
        shared_secret = self.private_key.exchange(ec.ECDH(), self.peer_public_key)
        self.shared_key = hashlib.sha256(shared_secret).digest()

# Example usage
if __name__ == "__main__":
    # Initialize ECDH instances for both Alice and Bob
    alice = ECDH()
    bob = ECDH()

    # Exchange public keys
    alice_public_key_bytes = alice.get_public_key_bytes()
    bob_public_key_bytes = bob.get_public_key_bytes()

    alice.load_peer_public_key(bob_public_key_bytes)
    bob.load_peer_public_key(alice_public_key_bytes)

    # Derive shared secrets
    alice.derive_shared_secret()
    bob.derive_shared_secret()

    # Print the shared secrets to verify
    print(f"Alice's derived shared secret: {alice.shared_key.hex()}")
    print(f"Bob's derived shared secret:   {bob.shared_key.hex()}")

    final_shared_key = alice.shared_key


Alice's derived shared secret: d360439918774d423e44b2d1a2f5f691cbcbc7e26475fb9f4f731b21fa84cdd9
Bob's derived shared secret:   d360439918774d423e44b2d1a2f5f691cbcbc7e26475fb9f4f731b21fa84cdd9


**Encryption of text using SIMON with generated key**

In [ ]:
from collections import deque
import struct
import time

class SimonCipher(object):
    """Simon Block Cipher Object"""

    # Z Arrays (stored bit reversed for easier usage)
    z0 = 0b01100111000011010100100010111110110011100001101010010001011111
    z1 = 0b01011010000110010011111011100010101101000011001001111101110001
    z2 = 0b11001101101001111110001000010100011001001011000000111011110101
    z3 = 0b11110000101100111001010001001000000111101001100011010111011011
    z4 = 0b11110111001001010011000011101000000100011011010110011110001011

    # valid cipher configurations stored:
    # block_size:{key_size:(number_rounds,z sequence)}
    __valid_setups = {32: {64: (32, z0)},
                      48: {72: (36, z0), 96: (36, z1)},
                      64: {96: (42, z2), 128: (44, z3)},
                      96: {96: (52, z2), 144: (54, z3)},
                      128: {128: (68, z2), 192: (69, z3), 256: (72, z4)}}

    __valid_modes = ['ECB', 'CTR', 'CBC', 'PCBC', 'CFB', 'OFB']

    def __init__(self, key, key_size=128, block_size=128, mode='ECB', init=0, counter=0):
        """
        Initialize an instance of the Simon block cipher.
        :param key: Int representation of the encryption key
        :param key_size: Int representing the encryption key in bits
        :param block_size: Int representing the block size in bits
        :param mode: String representing which cipher block mode the object should initialize with
        :param init: IV for CTR, CBC, PCBC, CFB, and OFB modes
        :param counter: Initial Counter value for CTR mode
        :return: None
        """

        # Setup block/word size
        try:
            self.possible_setups = self.__valid_setups[block_size]
            self.block_size = block_size
            self.word_size = self.block_size >> 1
        except KeyError:
            print('Invalid block size!')
            print('Please use one of the following block sizes:', [x for x in self.__valid_setups.keys()])
            raise

        # Setup Number of Rounds, Z Sequence, and Key Size
        try:
            self.rounds, self.zseq = self.possible_setups[key_size]
            self.key_size = key_size
        except KeyError:
            print('Invalid key size for selected block size!!')
            print('Please use one of the following key sizes:', [x for x in self.possible_setups.keys()])
            raise

        # Create Properly Sized bit mask for truncating addition and left shift outputs
        self.mod_mask = (2 ** self.word_size) - 1

        # Parse the given iv and truncate it to the block length
        try:
            self.iv = init & ((2 ** self.block_size) - 1)
            self.iv_upper = self.iv >> self.word_size
            self.iv_lower = self.iv & self.mod_mask
        except (ValueError, TypeError):
            print('Invalid IV Value!')
            print('Please Provide IV as int')
            raise

        # Parse the given Counter and truncate it to the block length
        try:
            self.counter = counter & ((2 ** self.block_size) - 1)
        except (ValueError, TypeError):
            print('Invalid Counter Value!')
            print('Please Provide Counter as int')
            raise

        # Check Cipher Mode
        try:
            position = self.__valid_modes.index(mode)
            self.mode = self.__valid_modes[position]
        except ValueError:
            print('Invalid cipher mode!')
            print('Please use one of the following block cipher modes:', self.__valid_modes)
            raise

        # Parse the given key and truncate it to the key length
        try:
            self.key = key & ((2 ** self.key_size) - 1)
        except (ValueError, TypeError):
            print('Invalid Key Value!')
            print('Please Provide Key as int')
            raise

        # Pre-compile key schedule
        m = self.key_size // self.word_size
        self.key_schedule = []

        # Create list of subwords from encryption key
        k_init = [((self.key >> (self.word_size * ((m-1) - x))) & self.mod_mask) for x in range(m)]

        k_reg = deque(k_init)  # Use queue to manage key subwords

        round_constant = self.mod_mask ^ 3  # Round Constant is 0xFFFF..FC

        # Generate all round keys
        for x in range(self.rounds):

            rs_3 = ((k_reg[0] << (self.word_size - 3)) + (k_reg[0] >> 3)) & self.mod_mask

            if m == 4:
                rs_3 = rs_3 ^ k_reg[2]

            rs_1 = ((rs_3 << (self.word_size - 1)) + (rs_3 >> 1)) & self.mod_mask

            c_z = ((self.zseq >> (x % 62)) & 1) ^ round_constant

            new_k = c_z ^ rs_1 ^ rs_3 ^ k_reg[m - 1]

            self.key_schedule.append(k_reg.pop())
            k_reg.appendleft(new_k)

    def encrypt_round(self, x, y, k):
        """
        Complete One Feistel Round
        :param x: Upper bits of current plaintext
        :param y: Lower bits of current plaintext
        :param k: Round Key
        :return: Upper and Lower ciphertext segments
        """

        # Generate all circular shifts
        ls_1_x = ((x >> (self.word_size - 1)) + (x << 1)) & self.mod_mask
        ls_8_x = ((x >> (self.word_size - 8)) + (x << 8)) & self.mod_mask
        ls_2_x = ((x >> (self.word_size - 2)) + (x << 2)) & self.mod_mask

        # XOR Chain
        xor_1 = (ls_1_x & ls_8_x) ^ y
        xor_2 = xor_1 ^ ls_2_x
        new_x = k ^ xor_2

        return new_x, x

    def decrypt_round(self, x, y, k):
        """Complete One Inverse Feistel Round
        :param x: Upper bits of current ciphertext
        :param y: Lower bits of current ciphertext
        :param k: Round Key
        :return: Upper and Lower plaintext segments
        """

        # Generate all circular shifts
        ls_1_y = ((y >> (self.word_size - 1)) + (y << 1)) & self.mod_mask
        ls_8_y = ((y >> (self.word_size - 8)) + (y << 8)) & self.mod_mask
        ls_2_y = ((y >> (self.word_size - 2)) + (y << 2)) & self.mod_mask

        # Inverse XOR Chain
        xor_1 = k ^ x
        xor_2 = xor_1 ^ ls_2_y
        new_x = (ls_1_y & ls_8_y) ^ xor_2

        return y, new_x

    def encrypt(self, plaintext):
        """
        Process new plaintext into ciphertext based on current cipher object setup
        :param plaintext: Int representing value to encrypt
        :return: Int representing encrypted value
        """
        try:
            b = (plaintext >> self.word_size) & self.mod_mask
            a = plaintext & self.mod_mask
        except TypeError:
            print('Invalid plaintext!')
            print('Please provide plaintext as int')
            raise

        if self.mode == 'ECB':
            b, a = self.encrypt_function(b, a)

        elif self.mode == 'CTR':
            true_counter = self.iv + self.counter
            d = (true_counter >> self.word_size) & self.mod_mask
            c = true_counter & self.mod_mask
            d, c = self.encrypt_function(d, c)
            b ^= d
            a ^= c
            self.counter += 1

        elif self.mode == 'CBC':
            b ^= self.iv_upper
            a ^= self.iv_lower
            b, a = self.encrypt_function(b, a)

            self.iv_upper = b
            self.iv_lower = a
            self.iv = (b << self.word_size) + a

        elif self.mode == 'PCBC':
            f, e = b, a
            b ^= self.iv_upper
            a ^= self.iv_lower
            b, a = self.encrypt_function(b, a)
            self.iv_upper = b ^ f
            self.iv_lower = a ^ e
            self.iv = (self.iv_upper << self.word_size) + self.iv_lower

        elif self.mode == 'CFB':
            d = self.iv_upper
            c = self.iv_lower
            d, c = self.encrypt_function(d, c)
            b ^= d
            a ^= c

            self.iv_upper = b
            self.iv_lower = a
            self.iv = (b << self.word_size) + a

        elif self.mode == 'OFB':
            d = self.iv_upper
            c = self.iv_lower
            d, c = self.encrypt_function(d, c)
            self.iv_upper = d
            self.iv_lower = c
            self.iv = (d << self.word_size) + c

            b ^= d
            a ^= c

        ciphertext = (b << self.word_size) + a

        return ciphertext

    def decrypt(self, ciphertext):
        """
        Process new ciphertest into plaintext based on current cipher object setup
        :param ciphertext: Int representing value to encrypt
        :return: Int representing decrypted value
        """
        try:
            b = (ciphertext >> self.word_size) & self.mod_mask
            a = ciphertext & self.mod_mask
        except TypeError:
            print('Invalid ciphertext!')
            print('Please provide ciphertext as int')
            raise

        if self.mode == 'ECB':
            a, b = self.decrypt_function(a, b)

        elif self.mode == 'CTR':
            true_counter = self.iv + self.counter
            d = (true_counter >> self.word_size) & self.mod_mask
            c = true_counter & self.mod_mask
            d, c = self.encrypt_function(d, c)
            b ^= d
            a ^= c
            self.counter += 1

        elif self.mode == 'CBC':
            f, e = b, a
            a, b = self.decrypt_function(a, b)
            b ^= self.iv_upper
            a ^= self.iv_lower

            self.iv_upper = f
            self.iv_lower = e
            self.iv = (f << self.word_size) + e

        elif self.mode == 'PCBC':
            f, e = b, a
            a, b = self.decrypt_function(a, b)
            b ^= self.iv_upper
            a ^= self.iv_lower
            self.iv_upper = (b ^ f)
            self.iv_lower = (a ^ e)
            self.iv = (self.iv_upper << self.word_size) + self.iv_lower

        elif self.mode == 'CFB':
            d = self.iv_upper
            c = self.iv_lower
            self.iv_upper = b
            self.iv_lower = a
            self.iv = (b << self.word_size) + a
            d, c = self.encrypt_function(d, c)
            b ^= d
            a ^= c

        elif self.mode == 'OFB':
            d = self.iv_upper
            c = self.iv_lower
            d, c = self.encrypt_function(d, c)
            self.iv_upper = d
            self.iv_lower = c
            self.iv = (d << self.word_size) + c

            b ^= d
            a ^= c

        plaintext = (b << self.word_size) + a

        return plaintext


    def encrypt_function(self, upper_word, lower_word):
        """
        Completes appropriate number of Simon Fiestel function to encrypt provided words
        Round number is based off of number of elements in key schedule
        upper_word: int of upper bytes of plaintext input
                    limited by word size of currently configured cipher
        lower_word: int of lower bytes of plaintext input
                    limited by word size of currently configured cipher
        x,y:        int of Upper and Lower ciphertext words
        """
        x = upper_word
        y = lower_word

        # Run Encryption Steps For Appropriate Number of Rounds
        for k in self.key_schedule:
             # Generate all circular shifts
            ls_1_x = ((x >> (self.word_size - 1)) + (x << 1)) & self.mod_mask
            ls_8_x = ((x >> (self.word_size - 8)) + (x << 8)) & self.mod_mask
            ls_2_x = ((x >> (self.word_size - 2)) + (x << 2)) & self.mod_mask

            # XOR Chain
            xor_1 = (ls_1_x & ls_8_x) ^ y
            xor_2 = xor_1 ^ ls_2_x
            y = x
            x = k ^ xor_2

        return x,y

    def decrypt_function(self, upper_word, lower_word):
        """
        Completes appropriate number of Simon Fiestel function to decrypt provided words
        Round number is based off of number of elements in key schedule
        upper_word: int of upper bytes of ciphertext input
                    limited by word size of currently configured cipher
        lower_word: int of lower bytes of ciphertext input
                    limited by word size of currently configured cipher
        x,y:        int of Upper and Lower plaintext words
        """
        x = upper_word
        y = lower_word

        # Run Encryption Steps For Appropriate Number of Rounds
        for k in reversed(self.key_schedule):
             # Generate all circular shifts
            ls_1_x = ((x >> (self.word_size - 1)) + (x << 1)) & self.mod_mask
            ls_8_x = ((x >> (self.word_size - 8)) + (x << 8)) & self.mod_mask
            ls_2_x = ((x >> (self.word_size - 2)) + (x << 2)) & self.mod_mask

            # XOR Chain
            xor_1 = (ls_1_x & ls_8_x) ^ y
            xor_2 = xor_1 ^ ls_2_x
            y = x
            x = k ^ xor_2

        return x,y

    def update_iv(self, new_iv):
        if new_iv:
            try:
                self.iv = new_iv & ((2 ** self.block_size) - 1)
                self.iv_upper = self.iv >> self.word_size
                self.iv_lower = self.iv & self.mod_mask
            except TypeError:
                print('Invalid Initialization Vector!')
                print('Please provide IV as int')
                raise
        return self.iv

def text_to_int(text):
    return int.from_bytes(text.encode(), 'big')

def int_to_text(n):
    return n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()

if __name__ == "__main__":

    start_time = time.time()

    key = int.from_bytes(final_shared_key, byteorder='big')
    simon = SimonCipher(key, key_size=64, block_size=32)

    # Read plaintext from a text file
    with open("image_base64.txt", "r") as f:
        plaintext = f.read()

    block_size = 4  # 4 characters correspond to 32-bit block size

    # Encrypt the plaintext in 4-character blocks
    encrypted_ints = []
    for i in range(0, len(plaintext), block_size):
        block = plaintext[i:i + block_size].ljust(block_size)
        plaintext_int = text_to_int(block)
        encrypted_int = simon.encrypt(plaintext_int)
        encrypted_ints.append(encrypted_int)

    # Write encrypted values to a text file
    with open("encrypted.txt", "w") as f:
        for encrypted_int in encrypted_ints:
            f.write(f"{hex(encrypted_int)}\n")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 0.25 seconds


**Decryption of encrypted text**

In [ ]:
# Read encrypted values from the file and decrypt them
decrypted_text = ""
with open("encrypted.txt", "r") as f:
    for line in f:
        encrypted_int = int(line.strip(), 16)
        decrypted_int = simon.decrypt(encrypted_int)
        decrypted_text += int_to_text(decrypted_int).rstrip('\x00')

# Write decrypted text to a text file
with open("decrypted.txt", "w") as f:
    f.write(decrypted_text)

**Decompression of sent text(Ignore this part for now)**

In [ ]:
output_compressed_path = 'compressed.txt'
output_decompressed_path = 'decompressed.txt'
huffman.decompress(output_compressed_path, output_decompressed_path)

'decompressed.txt'

**Text to Image conversion**

In [ ]:
import base64

# Read the base64 encoded data from the text file
with open("decrypted.txt", "r") as text_file:
    base64_image = text_file.read()

# Decode the Base64 encoded data into bytes
image_bytes = base64.b64decode(base64_image)

# Save the decoded bytes to a new image file
with open("decoded_image.jpeg", "wb") as image_file:
    image_file.write(image_bytes)

In [ ]:
# Relative Entropy

import math
from collections import Counter

def calculate_probability_distribution(file_path):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        data = file.read()

    # Calculate the frequency of each character in the file
    counter = Counter(data)
    total_chars = len(data)

    # Calculate the probability distribution
    probability_distribution = {char: count / total_chars for char, count in counter.items()}

    return probability_distribution

def calculate_kl_divergence(p, q):
    # Calculate the Kullback-Leibler divergence
    kl_divergence = 0.0
    for char in p:
        if char in q:
            kl_divergence += p[char] * math.log2(p[char] / q[char])
        else:
            kl_divergence += p[char] * math.log2(p[char] / (1e-10))  # small value to avoid log(0)
    return kl_divergence

# Example usage
real_text_file_path = 'image_base64.txt'  # Replace with your real text file path
encrypted_text_file_path = 'cipher_hex.txt'  # Replace with your encrypted text file path

p = calculate_probability_distribution(real_text_file_path)
q = calculate_probability_distribution(encrypted_text_file_path)

kl_divergence = calculate_kl_divergence(p, q)
print(f'KL Divergence (Relative Entropy): {kl_divergence}')

FileNotFoundError: [Errno 2] No such file or directory: 'cipher_hex.txt'